In [ ]:
%matplotlib inline

Kelimeleri sınıflandırmak için temel bir karakter düzeyinde RNN oluşturup eğiteceğiz. Karakter düzeyinde bir RNN kelimeleri bir dizi karakter olarak okur - her adımda bir tahmin ve "gizli durum" çıktısı verir ve önceki gizli durumunu her bir sonraki adıma besler. Son tahmini çıktı olarak alırız, yani kelimenin hangi sınıfa ait olduğunu.

Özellikle, 18 köken dilinden birkaç bin soyadı üzerinde eğitim vereceğiz ve bir ismin hangi dilden olduğunu yazımına göre tahmin edeceğiz:

::

$ python predict.py Hinton
(-0.47) Scottish
(-1.52) English
(-3.57) Irish

$ python predict.py Schmidhuber
(-0.19) German
(-2.48) Czech
(-2.68) Dutch

**Verilerin Hazırlanması**

.. Not:: Verileri buradan indirin <https://download.pytorch.org/tutorial/data.zip>_ ve geçerli dizine çıkarın.

data/names dizininde "[Language].txt" adlı 18 metin dosyası bulunur. Her dosya, çoğunlukla romanize edilmiş (ancak yine de Unicode'dan ASCII'ye dönüştürmemiz gerekiyor) satır başına bir ad olmak üzere bir sürü ad içerir.

Dil başına ad listelerinden oluşan bir sözlük elde edeceğiz, {language: [names ...]}. Genel değişkenler "category" ve "line" (bizim durumumuzda dil ve ad için) daha sonraki genişletilebilirlik için kullanılır.

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

# Belirtilen bir dosya yolundaki tüm dosyaları bulan bir fonksiyon
def findFiles(path): return glob.glob(path)

# 'data/names/' klasöründeki tüm .txt dosyalarını bul ve listele
print(findFiles('data/names/*.txt'))

import unicodedata
import string

# ASCII harfler ve bazı özel karakterler; isim verisi temizliği için kullanılacak
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)


# Unicode karakterleri ASCII'ye dönüştüren fonksiyon
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Örnek bir isimde Unicode'dan ASCII'ye dönüşümün çıktısını gösterir
print(unicodeToAscii('Ślusàrski'))

# Her bir kategoriye (ülkeye) ait isimlerin tutulacağı sözlük
category_lines = {}
# Tüm kategorilerin (ülkelerin) tutulacağı liste
all_categories = []


# Belirtilen dosyadan satırları okuyan fonksiyon
def readLines(filename):
    # Dosyadaki her satırı okuyup Unicode'dan ASCII'ye dönüştürür
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# 'data/names/' klasöründeki her dosya için işlemler
for filename in findFiles('data/names/*.txt'):
    # Dosya adından kategori adını çıkar (örneğin, İngilizce için 'English')
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)  # Kategori listesini güncelle
    lines = readLines(filename)      # Dosyadaki isimleri oku
    category_lines[category] = lines  # Kategoriye ait isimleri sözlüğe ekle

# Kategori (ülke) sayısını belirle
n_categories = len(all_categories)

Şimdi category_lines'ımız var, her kategoriyi (dil) bir satır listesine (isimler) eşleyen bir sözlük. Ayrıca all_categories'i (sadece bir dil listesi) ve n_categories'i daha sonra başvurmak üzere takip ettik.

In [ ]:
# 'Italian' anahtarı altında bulunan ilk 5 ismi yazdır
print(category_lines['Italian'][:5])

**İsimleri Tensörlere Dönüştürmek**

In [ ]:
Artık tüm isimleri düzenlediğimize göre, bunları herhangi bir şekilde kullanmak için Tensörlere dönüştürmemiz gerekiyor.

Tek bir harfi temsil etmek için, <1 x n_letters> boyutunda bir "one-hot vektör" kullanırız. One-hot vektör, geçerli harfin indeksindeki 1 hariç 0'larla doldurulur, örn. "b" = <0 1 0 0 0 ...>.

Bir kelime oluşturmak için bunlardan bir demetini <line_length x 1 x n_letters> boyutunda bir 2B matrise birleştiririz.

Bu ekstra 1 boyut, PyTorch'un her şeyin gruplar halinde olduğunu varsaymasından kaynaklanır - burada sadece 1'lik bir grup boyutu kullanıyoruz.

In [ ]:
import torch

# all_letters dizisinde belirtilen bir harfin indeksini bulan fonksiyon, örneğin "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Bir harfi <1 x n_letters> boyutunda bir tensöre çevirme fonksiyonu (örnek için)
# Bu tensör bir "one-hot" vektörüdür, sadece belirli bir harfin konumunda 1 olur
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)  # <1 x n_letters> boyutunda sıfırlarla dolu bir tensör oluşturur
    tensor[0][letterToIndex(letter)] = 1  # Harfin indeksine göre tensörde 1 olarak işaretler
    return tensor

# Bir metni <line_length x 1 x n_letters> boyutunda tensöre çevirme fonksiyonu
# Bu tensör, harfleri "one-hot" vektörleri olarak temsil eden bir dizi içerir
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)  # Her harf için sıfırlardan oluşan bir tensör oluşturur
    for li, letter in enumerate(line):  # Metindeki her harf için
        tensor[li][0][letterToIndex(letter)] = 1  # Harfin indeksine göre tensörde 1 olarak işaretler
    return tensor

# 'J' harfini "one-hot" tensör olarak yazdır
print(letterToTensor('J'))

# 'Jones' kelimesini tensör boyutunda yazdır
print(lineToTensor('Jones').size())

**Ağ Oluşturma**

Autograd'dan önce, Torch'ta yinelemeli bir sinir ağı oluşturmak, bir katmanın parametrelerini birkaç zaman adımı boyunca klonlamayı içeriyordu. Katmanlar, artık tamamen grafiğin kendisi tarafından işlenen gizli durum ve eğimleri tutuyordu. Bu, bir RNN'yi çok "saf" bir şekilde, düzenli ileri beslemeli katmanlar olarak uygulayabileceğiniz anlamına gelir.

Bu RNN modülü (çoğunlukla Torch kullanıcıları için PyTorch öğreticisinden kopyalanmıştır <http://pytorch.org/tutorials/beginner/former_torchies/ nn_tutorial.html#example-2-recurrent-net>__) yalnızca bir girdi ve gizli durum üzerinde çalışan 2 doğrusal katmandır ve çıktıdan sonra bir LogSoftmax katmanı vardır.

In [ ]:
import torch.nn as nn

# RNN sınıfını tanımlıyoruz, nn.Module sınıfını miras alıyor
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()  # nn.Module'ın özelliklerini RNN'e aktarır

        self.hidden_size = hidden_size  # Gizli katmanın boyutu

        # Giriş ve gizli durumun birleşiminden gizli duruma geçiş katmanı
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)

        # Giriş ve gizli durumun birleşiminden çıkışa geçiş katmanı
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

        # Çıkış katmanı için LogSoftmax aktivasyon fonksiyonu
        self.softmax = nn.LogSoftmax(dim=1)

    # İleri yayılım fonksiyonu, RNN'in giriş ve gizli durumu işleyerek çıktıyı üretmesini sağlar
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)  # Giriş ve gizli durumu birleştiriyoruz
        hidden = self.i2h(combined)               # Yeni gizli durumu hesaplıyoruz
        output = self.i2o(combined)               # Çıkış hesaplanıyor
        output = self.softmax(output)             # LogSoftmax ile normalize ediliyor
        return output, hidden                     # Çıktı ve yeni gizli durum döndürülüyor

    # Başlangıç gizli durumu oluşturma fonksiyonu (başlangıçta tüm değerler 0)
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

# Gizli katman boyutu
n_hidden = 128

# RNN modelini başlatıyoruz, giriş boyutu n_letters, çıkış boyutu n_categories
rnn = RNN(n_letters, n_hidden, n_categories)

Bu ağın bir adımını çalıştırmak için bir girdi (bizim durumumuzda, geçerli harf için Tensör) ve bir önceki gizli durumu (ilk başta sıfırlar olarak başlattığımız) geçirmemiz gerekir. Çıktıyı (her dilin olasılığı) ve bir sonraki gizli durumu (bir sonraki adım için sakladığımız) geri alacağız.

In [ ]:
# 'A' harfi için bir "one-hot" tensör oluşturuyoruz
input = letterToTensor('A')

# Gizli durumun başlangıç değeri olarak sıfırlardan oluşan bir tensör oluşturuyoruz
hidden = torch.zeros(1, n_hidden)

# RNN modeline 'A' harfi ve başlangıç gizli durumunu vererek çıktıyı ve bir sonraki gizli durumu elde ediyoruz
output, next_hidden = rnn(input, hidden)


Verimlilik uğruna her adım için yeni bir Tensör oluşturmak istemiyoruz, bu yüzden letterToTensor yerine lineToTensor kullanacağız ve dilimler kullanacağız. Bu, Tensörlerin toplu olarak önceden hesaplanmasıyla daha da iyileştirilebilir.

In [ ]:
# 'Albert' ismini temsil eden bir tensör oluşturuyoruz.
# Bu tensör, her harfi "one-hot" olarak temsil eden bir dizi tensörden oluşur.
input = lineToTensor('Albert')

# Gizli durumun başlangıç değeri olarak sıfırlardan oluşan bir tensör oluşturuyoruz
hidden = torch.zeros(1, n_hidden)

# İlk harfi (input[0]) ve gizli durumu RNN modeline vererek çıktıyı ve yeni gizli durumu alıyoruz
output, next_hidden = rnn(input[0], hidden)

# Çıktıyı ekrana yazdırıyoruz; bu çıktı, modelin 'A' harfi ve başlangıç gizli durumu ile oluşturduğu tahmini temsil eder
print(output)

Gördüğünüz gibi çıktı, her öğenin o kategorinin olasılığını gösterdiği <1 x n_categories> Tensörüdür (daha yüksek olması daha olasıdır).

**Eğitim**

**Eğitime Hazırlık**

Eğitime başlamadan önce birkaç yardımcı fonksiyon yapmalıyız. Birincisi, her kategorinin bir olasılığı olduğunu bildiğimiz ağın çıktısını yorumlamaktır. En büyük değerin dizinini elde etmek için Tensor.topk'u kullanabiliriz:

In [ ]:
# Modelin çıktısından kategori tahminini belirleyen fonksiyon
def categoryFromOutput(output):
    # En yüksek değeri ve onun indeksini buluyoruz (bu kategori tahminidir)
    top_n, top_i = output.topk(1)  # En yüksek olasılık değerine sahip olanı seçiyoruz
    category_i = top_i[0].item()   # İndeksi bir Python tamsayısına çeviriyoruz
    return all_categories[category_i], category_i  # Kategori adını ve indeksini döndürüyoruz

# Tahmin edilen kategori ve indeksini ekrana yazdırıyoruz
print(categoryFromOutput(output))


Ayrıca bir eğitim örneğini (bir isim ve dili) elde etmenin hızlı bir yolunu da isteyeceğiz:

In [ ]:
import random

# Listeden rastgele bir öğe seçen fonksiyon
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Rastgele bir eğitim örneği oluşturan fonksiyon
def randomTrainingExample():
    # Kategoriler arasından rastgele bir kategori seçiyoruz
    category = randomChoice(all_categories)

    # Seçilen kategoriye ait isimler arasından rastgele bir isim seçiyoruz
    line = randomChoice(category_lines[category])

    # Kategori için indeks tensörünü oluşturuyoruz (kategori numarasını uzunluk türünde tensor yapıyoruz)
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)

    # Seçilen ismi (line) tensöre çeviriyoruz
    line_tensor = lineToTensor(line)

    # Kategori adı, isim, kategori tensörü ve isim tensörünü döndürüyoruz
    return category, line, category_tensor, line_tensor

# 10 adet rastgele eğitim örneği oluşturup yazdırıyoruz
for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

**Ağın Eğitimi**

Şimdi bu ağı eğitmek için gereken tek şey ona bir sürü örnek göstermek, tahminlerde bulunmasını sağlamak ve yanlışsa bunu söylemek.

Kayıp fonksiyonu için nn.NLLLoss uygundur, çünkü RNN'nin son katmanı nn.LogSoftmax'tır.

In [ ]:
# Negatif Log-Likelihood (NLL) kayıp fonksiyonunu tanımlıyoruz
criterion = nn.NLLLoss()


Her eğitim döngüsü şunları yapacaktır:
-Giriş ve hedef tensörleri oluştur
-Sıfırlanmış bir başlangıç ​​gizli durumu oluştur
-Her harfi oku ve
-Gizli durumu bir sonraki harf için sakla
-Son çıktıyı hedefle karşılaştır
-Geri yayılım
-Çıktıyı ve kaybı döndür

In [ ]:
# Öğrenme oranını tanımlıyoruz; bu değer çok yüksek olursa model kararsızlaşabilir,
# çok düşük olursa model öğrenim sürecini etkili bir şekilde gerçekleştiremeyebilir
learning_rate = 0.005

# Modelin eğitimini gerçekleştiren fonksiyon
def train(category_tensor, line_tensor):
    # RNN için başlangıç gizli durumunu başlatıyoruz
    hidden = rnn.initHidden()

    # RNN modelinin gradyanlarını sıfırlıyoruz
    rnn.zero_grad()

    # Girdi tensöründeki her harf için RNN modelini çalıştırıyoruz
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    # Çıktıyı ve kategori tensörünü kullanarak kaybı hesaplıyoruz
    loss = criterion(output, category_tensor)

    # Kayıp değerine göre gradyanları hesaplıyoruz
    loss.backward()

    # Öğrenme oranı ile çarpılan gradyanları, parametre değerlerine ekliyoruz
    # Bu işlem, her parametrenin güncellenmesini sağlar
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    # Son çıktıyı ve kayıp değerini döndürüyoruz
    return output, loss.item()


Şimdi bunu bir sürü örnekle çalıştırmamız gerekiyor. Train fonksiyonu hem çıktıyı hem de kaybı döndürdüğünden tahminlerini yazdırabilir ve ayrıca çizim için kaybı takip edebiliriz. 1000'lerce örnek olduğundan yalnızca her print_every örneğini yazdırırız ve kaybın ortalamasını alırız.

In [ ]:
import time
import math

# Toplam iterasyon sayısı
n_iters = 100000
# Her 5000 iterasyonda çıktı vermek için ayarlama
print_every = 5000
# Her 1000 iterasyonda kayıp değerini grafiğe eklemek için ayarlama
plot_every = 1000

# Kayıpları takip etmek için bir liste
current_loss = 0
all_losses = []

# Geçen süreyi hesaplayan fonksiyon
def timeSince(since):
    now = time.time()  # Şu anki zamanı al
    s = now - since    # Başlangıç zamanından bu yana geçen süreyi hesapla
    m = math.floor(s / 60)  # Dakika cinsinden süreyi hesapla
    s -= m * 60  # Saniye cinsinden süreyi hesapla
    return '%dm %ds' % (m, s)  # Dakika ve saniyeyi döndür

start = time.time()  # Eğitim süresini başlat

# Eğitim döngüsü
for iter in range(1, n_iters + 1):
    # Rastgele bir eğitim örneği oluştur
    category, line, category_tensor, line_tensor = randomTrainingExample()
    # Modeli eğit ve kaybı hesapla
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss  # Geçerli kaybı güncelle

    # Belirli bir iterasyonda çıktı ver
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)  # Modelin tahminini al
        # Doğruluğu kontrol et
        correct = '✓' if guess == category else '✗ (%s)' % category
        # İterasyon numarasını, kaybı ve tahmini yazdır
        print('%d %d%% (%s) %.4f %s / %s %s' % (
            iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Geçerli kayıp ortalamasını kayıplar listesine ekle
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)  # Kayıp ortalamasını ekle
        current_loss = 0  # Geçerli kaybı sıfırla


**Sonuçların Çizilmesi**

all_losses'tan gelen tarihsel kaybın çizilmesi, ağ öğrenimini gösterir:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Yeni bir grafik oluştur
plt.figure()
# Kayıp değerlerini grafiğe çiz
plt.plot(all_losses)

# Eksenleri ayarlamak için ticker kullan
plt.xlabel('Iterasyon (her 1000\'de bir)')
plt.ylabel('Kayıp (Loss)')
plt.title('Eğitim Kaybı Grafiği')

# Y ekseninde sayıları daha iyi göstermek için bir format belirle
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))

# Grafiği göster
plt.show()


**Sonuçların Değerlendirilmesi**

Ağın farklı kategorilerde ne kadar iyi performans gösterdiğini görmek için, her gerçek dil (satırlar) için ağın hangi dili tahmin ettiğini (sütunlar) belirten bir karışıklık matrisi oluşturacağız. Karışıklık matrisini hesaplamak için bir grup örnek, train() eksi geri yayılımla aynı olan assess() ile ağda çalıştırılır.

In [ ]:
# Doğru tahminleri takip etmek için bir karmaşa matrisini başlat
confusion = torch.zeros(n_categories, n_categories)  # n_categories x n_categories boyutunda sıfırlardan oluşan bir matris
n_confusion = 10000  # Değerlendirme için kullanılacak örnek sayısı

# Bir satıra verilen çıktıyı döndürmek için fonksiyon
def evaluate(line_tensor):
    hidden = rnn.initHidden()  # RNN'in gizli durumunu başlat

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)  # Her harf için çıkışı hesapla

    return output  # Son çıkışı döndür

# Birçok örneği geçerek hangi tahminlerin doğru yapıldığını kaydet
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()  # Rastgele bir eğitim örneği al
    output = evaluate(line_tensor)  # Örneği değerlendir
    guess, guess_i = categoryFromOutput(output)  # Tahmin edilen kategoriyi al
    category_i = all_categories.index(category)  # Gerçek kategorinin indeksini bul
    confusion[category_i][guess_i] += 1  # Karmaşa matrisinde doğru tahmini güncelle

# Her satırı kendi toplamına bölerek normalize et
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()  # Satır toplamına böl

# Grafik ayarları
fig = plt.figure()  # Yeni bir figür oluştur
ax = fig.add_subplot(111)  # 1x1'lik bir ızgarada ilk alt grafiği oluştur
cax = ax.matshow(confusion.numpy())  # Karmaşa matrisini görselleştir
fig.colorbar(cax)  # Renk çubuğunu ekle

# Eksen ayarları
ax.set_xticklabels([''] + all_categories, rotation=90)  # X eksenindeki etiketleri ayarla
ax.set_yticklabels([''] + all_categories)  # Y eksenindeki etiketleri ayarla

# Her tikte etiket zorla
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))  # X eksenindeki her bir tike bir etiket yerleştir
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))  # Y eksenindeki her bir tike bir etiket yerleştir

# Grafiği göster
plt.show()


Ana eksenden hangi dilleri yanlış tahmin ettiğini gösteren parlak noktaları seçebilirsiniz, örneğin Korece için Çince ve İtalyanca için İspanyolca. Yunanca ile çok iyi, İngilizce ile ise çok kötü (belki de diğer dillerle örtüşmesinden dolayı).

**Kullanıcı Girişiyle Çalışma**

In [ ]:
def predict(input_line, n_predictions=3):
    # Kullanıcıdan alınan girdi satırını yazdır
    print('\n> %s' % input_line)
    with torch.no_grad():  # Gradyan hesaplamasını kapat (eğitim aşaması değil)
        output = evaluate(lineToTensor(input_line))  # Girdi satırını değerlendir

        # En yüksek N kategoriyi al
        topv, topi = output.topk(n_predictions, 1, True)  # En yüksek n_predictions değerini ve indekslerini al
        predictions = []  # Tahminleri saklamak için bir liste oluştur

        # Tahmin edilen kategorileri ve değerlerini yazdır
        for i in range(n_predictions):
            value = topv[0][i].item()  # Tahmin edilen değer
            category_index = topi[0][i].item()  # Tahmin edilen kategorinin indeksi
            print('(%.2f) %s' % (value, all_categories[category_index]))  # Değeri ve kategoriyi yazdır
            predictions.append([value, all_categories[category_index]])  # Tahminleri listeye ekle

# Örnek girdilerle tahminleri yap
predict('Dovesky')  # 'Dovesky' ismi için tahmin yap
predict('Jackson')  # 'Jackson' ismi için tahmin yap
predict('Satoshi')  # 'Satoshi' ismi için tahmin yap


Practical PyTorch deposundaki betiklerin son sürümleri <https://github.com/spro/practical-pytorch/tree/master/char-rnn-classification>__ yukarıdaki kodu birkaç dosyaya böldü:

-data.py (dosyaları yükler)
-model.py (RNN'i tanımlar)
-train.py (eğitimi çalıştırır)
-predict.py (predict()'i komut satırı argümanlarıyla çalıştırır)
-server.py (tahmini bottle.py ile JSON API olarak sunar)

Ağı eğitmek ve kaydetmek için train.py'yi çalıştırın.

Tahminleri görüntülemek için predict.py'yi bir adla çalıştırın:

$ python predict.py Hazaki

(-0.42) Japanese

(-1.39) Polish

(-3.51) Czech

server.py'ı çalıştırın ve tahminlerin JSON çıktısını almak için http://localhost:5533/Adınız adresine gidin.